In [ ]:
import os
import feast

actual_version = feast.__version__
assert actual_version == os.environ.get("FEAST_VERSION"), (
    f"❌ Feast version mismatch. Expected: {os.environ.get('FEAST_VERSION')}, Found: {actual_version}"
)
print(f"✅ Successfully installed Feast version: {actual_version}")

In [ ]:
%cd /opt/app-root/src/feature_repo
!ls -l

In [ ]:
!cat /opt/app-root/src/feature_repo/feature_store.yaml

In [ ]:
!mkdir -p data
!wget -O data/city_wikipedia_summaries_with_embeddings.parquet https://raw.githubusercontent.com/opendatahub-io/feast/master/examples/rag/feature_repo/data/city_wikipedia_summaries_with_embeddings.parquet

In [ ]:
import pandas as pd 

df = pd.read_parquet("./data/city_wikipedia_summaries_with_embeddings.parquet")
df['vector'] = df['vector'].apply(lambda x: x.tolist())
embedding_length = len(df['vector'][0])
assert embedding_length == 384, f"❌ Expected vector length 384, but got {embedding_length}"
print(f'embedding length = {embedding_length}')

In [ ]:
from IPython.display import display

display(df.head())

In [ ]:
!pip install -q pymilvus[milvus_lite] transformers torch

In [ ]:
import subprocess

# Run `feast apply` and capture output
result = subprocess.run(["feast", "apply"], capture_output=True, text=True)

# Combine stdout and stderr in case important info is in either
output = result.stdout + result.stderr

# Print full output for debugging (optional)
print(output)

# Expected substrings to validate
expected_messages = [
    "Applying changes for project rag",
    "Connecting to Milvus in local mode",
    "Deploying infrastructure for city_embeddings"
]

# Validate all expected messages are in output
for msg in expected_messages:
    assert msg in output, f"❌ Expected message not found: '{msg}'"

print("✅ All expected messages were found in the output.")


In [ ]:
from datetime import datetime
from feast import FeatureStore

store = FeatureStore(repo_path=".")

In [ ]:
import io
import sys

# Capture stdout
captured_output = io.StringIO()
sys_stdout_backup = sys.stdout
sys.stdout = captured_output

# Call the function
store.write_to_online_store(feature_view_name='city_embeddings', df=df)

# Restore stdout
sys.stdout = sys_stdout_backup

# Get the output
output_str = captured_output.getvalue()

# Expected message
expected_msg = "Connecting to Milvus in local mode using data/online_store.db"

# Validate
assert expected_msg in output_str, f"❌ Expected message not found.\nExpected: {expected_msg}\nActual Output:\n{output_str}"

print("✅ Output message validated successfully.")


In [ ]:
# List batch feature views
batch_fvs = store.list_batch_feature_views()

# Print the number of batch feature views
print("Number of batch feature views:", len(batch_fvs))

# Assert that the result is an integer and non-negative
assert isinstance(len(batch_fvs), int), "Result is not an integer"
assert len(batch_fvs) >= 0, "Feature view count is negative"

print("Feature views listed correctly ✅")

In [ ]:
from feast import FeatureStore

# Initialize store (if not already)
store = FeatureStore(repo_path=".")  # Adjust path if necessary

# Retrieve the feature view
fv = store.get_feature_view("city_embeddings")

# Assert name
assert fv.name == "city_embeddings", "Feature view name mismatch"

# Assert entities
assert fv.entities == ["item_id"], f"Expected entities ['item_id'], got {fv.entities}"

# Assert feature names and vector index settings
feature_names = [f.name for f in fv.features]
assert "vector" in feature_names, "Missing 'vector' feature"
assert "state" in feature_names, "Missing 'state' feature"
assert "sentence_chunks" in feature_names, "Missing 'sentence_chunks' feature"
assert "wiki_summary" in feature_names, "Missing 'wiki_summary' feature"

# Assert 'vector' feature is a vector index with COSINE metric
vector_feature = next(f for f in fv.features if f.name == "vector")
assert vector_feature.vector_index, "'vector' feature is not indexed"
assert vector_feature.vector_search_metric == "COSINE", "Expected COSINE search metric for 'vector'"

print("All assertions passed ✅")

In [ ]:
from feast.entity import Entity
from feast.types import ValueType
entity = Entity(
    name="item_id1",
    value_type=ValueType.INT64,
    description="test id",
    tags={"team": "feast"},
)
store.apply(entity)
assert any(e.name == "item_id1" for e in store.list_entities())
print("Entity added ✅")

In [ ]:
entity_to_delete = store.get_entity("item_id1")

store.apply(
    objects=[],
    objects_to_delete=[entity_to_delete],
    partial=False
)

# Validation after deletion
assert not any(e.name == "item_id1" for e in store.list_entities())
print("Entity deleted ✅")

In [ ]:
# List batch feature views
batch_fvs = store.list_batch_feature_views()
assert len(batch_fvs) == 1

# Print count
print(f"Found {len(batch_fvs)} batch feature view(s) ✅")


In [ ]:
pymilvus_client = store._provider._online_store._connect(store.config)
COLLECTION_NAME = pymilvus_client.list_collections()[0]

milvus_query_result = pymilvus_client.query(
    collection_name=COLLECTION_NAME,
    filter="item_id == '0'",
)
pd.DataFrame(milvus_query_result[0]).head()

In [ ]:
import torch
import torch.nn.functional as F
from feast import FeatureStore
from pymilvus import MilvusClient, DataType, FieldSchema
from transformers import AutoTokenizer, AutoModel
from example_repo import city_embeddings_feature_view, item

TOKENIZER = "sentence-transformers/all-MiniLM-L6-v2"
MODEL = "sentence-transformers/all-MiniLM-L6-v2"

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[
        0
    ]  # First element of model_output contains all token embeddings
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

def run_model(sentences, tokenizer, model):
    encoded_input = tokenizer(
        sentences, padding=True, truncation=True, return_tensors="pt"
    )
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    sentence_embeddings = mean_pooling(model_output, encoded_input["attention_mask"])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings

In [ ]:
question = "Which city has the largest population in New York?"

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)
model = AutoModel.from_pretrained(MODEL)
query_embedding = run_model(question, tokenizer, model)
query = query_embedding.detach().cpu().numpy().tolist()[0]

In [ ]:
from IPython.display import display

# Retrieve top k documents
context_data = store.retrieve_online_documents_v2(
    features=[
        "city_embeddings:vector",
        "city_embeddings:item_id",
        "city_embeddings:state",
        "city_embeddings:sentence_chunks",
        "city_embeddings:wiki_summary",
    ],
    query=query,
    top_k=3,
    distance_metric='COSINE',
).to_df()
display(context_data)

In [ ]:
def format_documents(context_df):
    output_context = ""
    unique_documents = context_df.drop_duplicates().apply(
        lambda x: "City & State = {" + x['state'] +"}\nSummary = {" + x['wiki_summary'].strip()+"}",
        axis=1,
    )
    for i, document_text in enumerate(unique_documents):
        output_context+= f"****START DOCUMENT {i}****\n{document_text.strip()}\n****END DOCUMENT {i}****"
    return output_context

In [ ]:
RAG_CONTEXT = format_documents(context_data[['state', 'wiki_summary']])
print(RAG_CONTEXT)

In [ ]:
FULL_PROMPT = f"""
You are an assistant for answering questions about states. You will be provided documentation from Wikipedia. Provide a conversational answer.
If you don't know the answer, just say "I do not know." Don't make up an answer.

Here are document(s) you should use when answer the users question:
{RAG_CONTEXT}
"""

In [ ]:
!pip install openai

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": FULL_PROMPT},
        {"role": "user", "content": question}
    ],
)

In [ ]:
# The expected output
expected_output = (
    "New York City"
)

# Actual output from response
actual_output = '\n'.join([c.message.content.strip() for c in response.choices])

# Validate
assert expected_output in actual_output, f"❌ Output mismatch:\nExpected: {expected_output}\nActual: {actual_output}"

print("✅ Output matches expected response.")